In [1]:
import gzip
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm.auto import tqdm
import seaborn as sns
from collections import defaultdict

freebase_path = "/Users/alexdremov/Downloads/freebase-rdf-latest.gz"

In [2]:
char_processed = pd.read_csv(
    "../../data/MovieSummaries/character_processed.csv",
    parse_dates=["actor_date_of_birth", "movie_release_date"],
)

char_processed = char_processed.rename(columns={
    'Wikipedia movie ID': "wikipedia_movie_id",
    'Freebase movie ID': "fb_movie_id",
    'Character name': "character_name",
    'Actor gender': "actor_gender",
    'Actor height (in meters)': "actor_height",
    'Actor ethnicity (Freebase ID)': "fb_actor_eth_id",
    'Actor name': "actor_name",
    'Freebase character/actor map ID': "fb_char_actor_map_id",
    'Freebase character ID': "fb_char_id",
    'Freebase actor ID': "fb_actor_id",
})

char_processed

,wikipedia_movie_id,fb_movie_id,character_name,actor_gender,actor_height,fb_actor_eth_id,actor_name,fb_char_actor_map_id,fb_char_id,fb_actor_id,actor_date_of_birth,movie_release_date,ethn_name,race
0,975900,/m/03vyhn,Akooshay,F,1.620,NaN,Wanda De Jesus,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7,1958-08-26,2001-08-24,NaN,NaN
1,975900,/m/03vyhn,Lieutenant Melanie Ballard,F,1.780,/m/044038p,Natasha Henstridge,/m/0jys3m,/m/0bgchn4,/m/0346l4,1974-08-15,2001-08-24,NaN,NaN
2,975900,/m/03vyhn,Desolation Williams,M,1.727,/m/0x67,Ice Cube,/m/0jys3g,/m/0bgchn_,/m/01vw26l,1969-06-15,2001-08-24,African American,African
3,975900,/m/03vyhn,Sgt Jericho Butler,M,1.750,NaN,Jason Statham,/m/02vchl6,/m/0bgchnq,/m/034hyc,1967-09-12,2001-08-24,NaN,NaN
4,975900,/m/03vyhn,Bashira Kincaid,F,1.650,NaN,Clea DuVall,/m/02vbb3r,/m/0bgchp9,/m/01y9xg,1977-09-25,2001-08-24,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334982,913762,/m/03pcrp,UN Spacy Commander,M,NaN,NaN,Sonny Byrkett,/m/0kr407w,/m/0kr407_,/m/0gn4bz,1954-01-01,1992-05-21,NaN,NaN
334983,913762,/m/03pcrp,Silvie Gena,F,NaN,NaN,Susan Byrkett,/m/0kr40b9,/m/0kr40bf,/m/0gn4nd,1958-01-01,1992-05-21,NaN,NaN
334984,913762,/m/03pcrp,Elensh,F,NaN,NaN,Dorothy Elias-Fahn,/m/0kr406c,/m/0kr406h,/m/0b_vcv,1970-05-01,1992-05-21,NaN,NaN
334985,913762,/m/03pcrp,Hibiki,M,NaN,NaN,Jonathan Fahn,/m/0kr405_,/m/0kr4090,/m/0bx7_j,1965-04-12,1992-05-21,NaN,NaN


In [3]:
movies_processed = pd.read_csv(
    "../../data/MovieSummaries/movie_processed.csv",
    parse_dates=["movie_release_date"],
)

movies_processed.rename(columns={
    "Wikipedia movie ID": "wikipedia_movie_id",
    "Freebase movie ID": "fb_movie_id",
    "Movie name": "movie_name",
    "Movie box office revenue": "revenue",
    "Movie runtime": "movie_runtime",
}, inplace=True)
movies_processed

,wikipedia_movie_id,fb_movie_id,movie_name,revenue,movie_runtime,languages,movie_release_date,countries_old,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,14010832.0,98.0,['english'],2001-08-24,['United States of America'],['United States of America'],"['Adventure', 'Horror', 'Action', 'Thriller', ..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,NaN,95.0,['english'],2000-02-16,['United States of America'],['United States of America'],"['Crime', 'Drama', 'Mystery', 'Biography']"
2,28463795,/m/0crgdbh,Brun bitter,NaN,83.0,['norwegian'],1988-01-01,['Norway'],['Norway'],"['Crime', 'Drama']"
3,9363483,/m/0285_cd,White Of The Eye,NaN,110.0,['english'],1987-01-01,['United Kingdom'],['United Kingdom'],"['Erotic', 'Thriller']"
4,261236,/m/01mrr1,A Woman in Flames,NaN,106.0,['german'],1983-01-01,['Germany'],['Germany'],['Drama']
...,...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,NaN,120.0,['english'],2011-03-19,['United States of America'],['United States of America'],['Drama']
81737,34980460,/m/0g4pl34,Knuckle,NaN,96.0,['english'],2011-01-21,"['Ireland', 'United Kingdom']","['Ireland', 'United Kingdom']","['Documentary', 'Drama', 'Biography']"
81738,9971909,/m/02pygw1,Another Nice Mess,NaN,66.0,['english'],1972-09-22,['United States of America'],['United States of America'],"['Comedy', 'Satire']"
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,NaN,150.0,['japanese'],1992-05-21,['Japan'],['Japan'],"['Drama', 'Adventure', 'Short Film', 'Cartoon'..."


In [5]:
interesting_ids = (
    set(char_processed.fb_movie_id) |
    set(char_processed.fb_actor_eth_id) |
    set(char_processed.fb_char_actor_map_id) |
    set(char_processed.fb_char_id) |
    set(char_processed.fb_actor_id) |
    set(movies_processed.fb_movie_id)
)
interesting_ids = set([
    '<http://rdf.freebase.com/ns/' + i.lstrip('/').replace('/', '.') + '>'
    for i in interesting_ids
    if isinstance(i, str)
])

len(interesting_ids), next(iter(interesting_ids))

(616271, '<http://rdf.freebase.com/ns/m.0j8nyhd>')

In [6]:
import re

def extract_urls(text):
    # Define the regular expression pattern
    pattern = r"<http://rdf\.freebase\.com/ns/[^\s]+>"
    urls = re.findall(pattern, text)
    return urls


extract_urls("sdjkfh <http://rdf.freebase.com/ns/base.type_ontology.non_agent> sdjknfhls")

['<http://rdf.freebase.com/ns/base.type_ontology.non_agent>']

In [7]:
extracted_lines = 0
with gzip.open(freebase_path, 'r') as fin, gzip.open('extracted-all.gz', 'w') as fout:
    progress = tqdm(fin, total=int(3.14 * 1e9))
    for line in progress:
        triplets = line.decode('utf-8').split('\t', 1)
        if triplets[0] in interesting_ids:
            extracted_lines += 1
            progress.set_description(f"{extracted_lines = }", refresh=False)
            fout.write(line)

  0%|          | 0/3140000000 [00:00<?, ?it/s]

In [9]:
single_link_ids = set()

with gzip.open('extracted-all.gz', 'r') as fin:
    progress = tqdm(fin, total=26196949)
    for line in progress:
        triplets = line.decode('utf-8')
        single_link_ids |= set(extract_urls(line.decode('utf-8')))
        progress.set_description(f"{len(single_link_ids) = }", refresh=False)

  0%|          | 0/26196949 [00:00<?, ?it/s]

In [10]:
full_ids = single_link_ids | interesting_ids
len(full_ids)

5328093

In [ ]:
extracted_lines = 0
with gzip.open(freebase_path, 'r') as fin, gzip.open('extracted-all-linked.gz', 'w') as fout:
    progress = tqdm(fin, total=int(3.14 * 1e9), mininterval=2.0)
    for line in progress:
        triplets = line.decode('utf-8').split('\t', 1)
        if triplets[0] in full_ids:
            extracted_lines += 1
            progress.set_description(f"{extracted_lines = }", refresh=False)
            fout.write(line)

  0%|          | 0/3140000000 [00:00<?, ?it/s]

In [32]:
def read_extracted(target, path):
    with gzip.open(path, 'r') as fin:
        for line in tqdm(fin, mininterval=2.0):
            triplets = line.decode('utf-8').split('\t', 1)
            key = triplets[0].lstrip('<http://rdf.freebase.com/ns/').rstrip('>')
            if key not in target:
                target[key] = list()
            target[key].append(triplets[1:])

In [3]:
import gzip, dill

In [24]:
with gzip.open("extracted_freebase.dill.gz", 'r') as file:
    movies_actors = dill.load(file)

In [12]:
person_related_ids = set()

needle = 'people.person.'.encode('utf-8')

with gzip.open('extracted.gz', 'r') as fin:
    progress = tqdm(fin, total=289924448, mininterval=2.0)
    for line in progress:
        if needle in line:
            person_related_ids |= set(extract_urls(line.decode('utf-8')))

  0%|          | 0/289924448 [00:00<?, ?it/s]

In [13]:
len(person_related_ids)

610405

In [14]:
extracted_data = defaultdict(list)

with gzip.open('extracted.gz', 'r') as fin:
    progress = tqdm(fin, total=289924448, mininterval=2.0)
    for line in progress:
        line = line.decode('utf-8')
        triplets = line.split('\t', 2)
        if triplets[0] in person_related_ids:
            progress.set_description(f"{len(extracted_data) = }", refresh=False)
            extracted_data[triplets[0]].append(triplets[1:])

  0%|          | 0/289924448 [00:00<?, ?it/s]

In [5]:
import gzip, dill

with gzip.open("extracted_freebase.dill.gz", 'r') as file:
    movies_actors = dill.load(file)

with gzip.open("extracted_personas.dill.gz", 'r') as file:
    personas = dill.load(file)

In [38]:
def extract_label(entries):
    for predicate, value in entries:
        if predicate == '<http://www.w3.org/2000/01/rdf-schema#label>' and '@en' in value:
            return value.replace('@en', '').strip('\t"\n. ')
    return None

def extract_actor_info(fb_id):
    fb_id = fb_id.strip('/').replace('/', '.')
    fb_id = f"<http://rdf.freebase.com/ns/{fb_id}>"

    if fb_id not in movies_actors:
        return None

    info = movies_actors[fb_id]

    result = dict(
        education=False,
        count_guest_roles=sum(i[0] == '<http://rdf.freebase.com/ns/tv.tv_actor.guest_roles>' for i in info),
        places_lived=sum(i[0] == '<http://rdf.freebase.com/ns/people.person.places_lived>' for i in info),
        spouses=sum(i[0] == '<http://rdf.freebase.com/ns/people.person.spouse_s>' for i in info),
        children=sum(i[0] == '<http://rdf.freebase.com/ns/people.person.children>' for i in info),
        professions_num=sum(i[0] == '<http://rdf.freebase.com/ns/people.person.profession>' for i in info),
        professions=[]
    )

    for predicate, value in info:
        value = value.strip('.\t \n')

        if predicate == '<http://rdf.freebase.com/ns/people.person.date_of_birth>':
            result['date_of_birth'] = value.split("^^")[0].strip('"')

        if predicate == '<http://rdf.freebase.com/ns/people.person.gender>':
            result['gender'] = extract_label(personas[value])

        if predicate == '<http://rdf.freebase.com/ns/people.person.height_meters>':
            result['height'] = value.strip('" .')

        if predicate == '<http://rdf.freebase.com/ns/people.person.nationality>':
            result['nationality'] = extract_label(personas[value])

        if predicate == '<http://rdf.freebase.com/ns/people.person.profession>':
            result['professions'].append(extract_label(personas[value]))

        if predicate == '<http://rdf.freebase.com/ns/people.person.weight_kg>':
            result['weight'] = value.strip('" .')

        if predicate == '<http://rdf.freebase.com/ns/people.person.education>':
            result['education'] = True

        if predicate == '<http://rdf.freebase.com/ns/people.person.place_of_birth>':
            result['place_of_birth'] = extract_label(personas[value])

        if predicate == '<http://rdf.freebase.com/ns/people.person.religion>':
            result['religion'] = extract_label(personas[value])
    return result

In [39]:
all_actors = set(char_processed.fb_actor_id)

In [40]:
extracted_actors_data = dict()

for actor_id in tqdm(all_actors):
    extracted_actors_data[actor_id] = extract_actor_info(actor_id)

  0%|          | 0/58176 [00:00<?, ?it/s]

In [41]:
# import json

# with open('actors_data_freebase.json', 'w') as file:
#     json.dump(extracted_actors_data, file)

In [19]:
with open('actors_data_freebase.json') as file:
    extracted_actors_data = json.load(file)

In [42]:
for i, val in extracted_actors_data.items():
    if val is not None:
        val['id'] = i

In [43]:
df = pd.DataFrame(list(filter(lambda x: x is not None, extracted_actors_data.values())))
df

,education,count_guest_roles,places_lived,spouses,children,professions_num,professions,place_of_birth,date_of_birth,gender,nationality,id,height,religion,weight
0,False,2,1,2,2,2,"[Comedian""-GB, Actor]","Jerusalem""-GB",1929-11-04,Male,Israel,/m/02qy84q,NaN,NaN,NaN
1,False,22,0,0,0,1,[Actor],Burbank,1993-01-14,Male,United States of America,/m/06zmx3x,NaN,NaN,NaN
2,False,0,0,0,1,1,[Actor],Pelhřimov,1923-06-19,Male,Czech Republic,/m/0b_h6gk,NaN,NaN,NaN
3,False,0,0,0,0,1,[Actor],Dumfries,1851-10,Male,Scotland,/m/0lnbcrb,NaN,NaN,NaN
4,True,1,0,0,0,1,[Actor],"Denton, Texas",1945-08-09,Male,United States of America,/m/0k7072,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57571,False,0,0,0,0,1,[Actor],California,1974-11-19,Female,United States of America,/m/0glptws,NaN,NaN,NaN
57572,False,48,0,0,0,2,"[Actor, Journalist]",Terre Haute,1903-08-13,Male,United States of America,/m/0fqqcz,NaN,NaN,NaN
57573,False,0,0,0,0,0,[],Lincoln,1992-04-04,Female,NaN,/m/09v7d26,NaN,NaN,NaN
57574,False,0,0,1,0,0,[],Vienna,1913-01-09,Male,NaN,/m/0dlhch,NaN,NaN,NaN


In [44]:
df.to_csv('actors_freebase.csv')

In [50]:
all_chars = set(char_processed.fb_char_id) | set(char_processed.fb_char_actor_map_id)

interesting_ids = set([
    '<http://rdf.freebase.com/ns/' + i.lstrip('/').replace('/', '.') + '>'
    for i in all_chars
    if isinstance(i, str)
])

In [55]:
extracted_data = defaultdict(list)

with gzip.open('extracted.gz', 'r') as fin:
    progress = tqdm(fin, total=289924448, mininterval=2.0)
    for line in progress:
        line = line.decode('utf-8')
        triplets = line.split('\t', 2)
        if triplets[0] in interesting_ids:
            progress.set_description(f"{len(extracted_data) = }", refresh=False)
            extracted_data[triplets[0]].append(triplets[1:])

  0%|          | 0/289924448 [00:00<?, ?it/s]

In [56]:
# with gzip.open("extracted_chars.dill.gz", 'w') as file:
#     dill.dump(extracted_data, file)

In [57]:
with gzip.open("extracted_chars.dill.gz", 'r') as file:
    chars_data = dill.load(file)

In [73]:
def extract_character_info(fb_id):
    fb_id = fb_id.strip('/').replace('/', '.')
    fb_id = f"<http://rdf.freebase.com/ns/{fb_id}>"

    if fb_id not in movies_actors:
        return None

    info = chars_data[fb_id]

    result = dict()

    for predicate, value in info:
        value: str = value.strip('\t.\n ')
        print(predicate, value)
        if predicate == '<http://rdf.freebase.com/ns/common.topic.description>' and value.endswith('@en'):
            result['description'] = value.replace('@en', '').strip('".')
    return result

In [74]:
extracted_chars_data = dict()

for char_id in char_processed.fb_char_id:
    if isinstance(char_id, str):
        extracted_chars_data[char_id] = extract_character_info(char_id)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

In [77]:
movies_processed.fb_movie_id

0         /m/03vyhn
1         /m/08yl5d
2        /m/0crgdbh
3        /m/0285_cd
4         /m/01mrr1
            ...    
81736    /m/0j7hxnt
81737    /m/0g4pl34
81738    /m/02pygw1
81739     /m/03pcrp
81740    /m/02w7zz8
Name: fb_movie_id, Length: 81741, dtype: object